In [22]:
import json
import os
import math
import librosa

DATASET_PATH = "genres_reduced"
JSON_PATH = "data.json"
SAMPLE_RATE = 22050
TRACK_DURATION = 30 # measured in seconds
SAMPLES_PER_TRACK = SAMPLE_RATE * TRACK_DURATION


def save_mfcc(dataset_path, json_path, num_mfcc=13, n_fft=2048, hop_length=512, num_segments=5):
    
    # dictionary to store mapping, labels, and MFCCs
    data = {
        "mapping": [],
        "labels": [],
        "mfcc": []
    }

    samples_per_segment = int(SAMPLES_PER_TRACK / num_segments)
    num_mfcc_vectors_per_segment = math.ceil(samples_per_segment / hop_length)

    # loop through all genre sub-folder
    for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):

        # ensure we're processing a genre sub-folder level
        if dirpath is not dataset_path:

            # save genre label (i.e., sub-folder name) in the mapping
            semantic_label = dirpath.split("/")[-1]
            data["mapping"].append(semantic_label)
            print("\nProcessing: {}".format(semantic_label))

            # process all audio files in genre sub-dir
            for f in filenames:

		# load audio file
                file_path = os.path.join(dirpath, f)
                signal, sample_rate = librosa.load(file_path, sr=SAMPLE_RATE)

                # process all segments of audio file
                for d in range(num_segments):

                    # calculate start and finish sample for current segment
                    start = samples_per_segment * d
                    finish = start + samples_per_segment

                    # extract mfcc
                    mfcc = librosa.feature.mfcc(signal[start:finish], sample_rate, n_mfcc=num_mfcc, n_fft=n_fft, hop_length=hop_length)
                    mfcc = mfcc.T

                    # store only mfcc feature with expected number of vectors
                    if len(mfcc) == num_mfcc_vectors_per_segment:
                        data["mfcc"].append(mfcc.tolist())
                        data["labels"].append(i-1)
                        print("{}, segment:{}".format(file_path, d+1))

    # save MFCCs to json file
    with open(json_path, "w") as fp:
        json.dump(data, fp, indent=4)

In [23]:
save_mfcc(DATASET_PATH, JSON_PATH, num_segments=10)


Processing: genres_reduced\blues
genres_reduced\blues\blues.00000.wav, segment:1
genres_reduced\blues\blues.00000.wav, segment:2
genres_reduced\blues\blues.00000.wav, segment:3
genres_reduced\blues\blues.00000.wav, segment:4
genres_reduced\blues\blues.00000.wav, segment:5
genres_reduced\blues\blues.00000.wav, segment:6
genres_reduced\blues\blues.00000.wav, segment:7
genres_reduced\blues\blues.00000.wav, segment:8
genres_reduced\blues\blues.00000.wav, segment:9
genres_reduced\blues\blues.00000.wav, segment:10

Processing: genres_reduced\classical
genres_reduced\classical\classical.00000.wav, segment:1
genres_reduced\classical\classical.00000.wav, segment:2
genres_reduced\classical\classical.00000.wav, segment:3
genres_reduced\classical\classical.00000.wav, segment:4
genres_reduced\classical\classical.00000.wav, segment:5
genres_reduced\classical\classical.00000.wav, segment:6
genres_reduced\classical\classical.00000.wav, segment:7
genres_reduced\classical\classical.00000.wav, segment:8